In [53]:
import pandas as pd

def IoTcode2vulBERT(csv_input, csv_output):
    df = pd.read_csv(csv_input)
    """ convert data to vulBERT format """
    if 'is_vul' in list(df.columns):
        df = df[['code', 'is_vul']]
        df = df.rename(columns={'is_vul':'label'})
        df['label'] = df.label.astype(int)

    df.to_csv(csv_output)
    return df

dataset = 'raspberryZip'
csv_input = f"../data/{dataset}_function.csv"
csv_vulbert =f"../data/{dataset}_function-VulBERT.csv"
# csv = '../VulBERTa/data/finetune/d2a/function/d2a_lbv1_function_dev.csv'

df = IoTcode2vulBERT(
    csv_input=csv_input,
    csv_output=csv_vulbert
    )

In [54]:
dataset = 'raspberryZip'
flaw = f"../data/{dataset}_statement.csv"
metrics = f"../data/{dataset}_function.csv"
binary = f"../data/{dataset}_method_binary.csv"

df_stat  = pd.read_csv(flaw)
df_fun = pd.read_csv(metrics)

print(f'Fun statistics: \n{df_fun.isVul.value_counts()}')
print(f'\nStat statistics: \n{df_stat.cwe.value_counts()}')

Fun statistics: 
1    2212
Name: isVul, dtype: int64

Stat statistics: 
Benign              5031
CWE-78                91
CWE-119!/CWE-120      56
CWE-126               55
CWE-120               53
CWE-120, CWE-20       27
CWE-362                7
CWE-134                1
CWE-unknown            1
Name: cwe, dtype: int64


In [56]:
df_fun.drop_duplicates(subset=['filename', 'long_name', 'start_line', 'end_line', 'cwe'], keep='last').reset_index(drop=True)

,cyclomatic_complexity,nloc,token_count,name,long_name,start_line,end_line,filename,code,fun_name,content,isVul,cwe
0,11,38,212,ftrace_record_recursion,"ftrace_record_recursion( unsigned long ip , un...",25,115,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,"ftrace_record_recursion( unsigned long ip , un...",ftrace_record_recursion,[('\tif (index >= CONFIG_FTRACE_RECORD_RECURSI...,1,CWE-unknown
1,3,15,87,recursed_function_seq_start,recursed_function_seq_start( struct seq_file *...,121,139,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,recursed_function_seq_start( struct seq_file *...,recursed_function_seq_start,NaN,1,Benign
2,2,8,52,recursed_function_seq_next,recursed_function_seq_next( struct seq_file * ...,141,150,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,recursed_function_seq_next( struct seq_file * ...,recursed_function_seq_next,NaN,1,Benign
3,1,5,24,recursed_function_seq_stop,recursed_function_seq_stop( struct seq_file * ...,152,156,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,recursed_function_seq_stop( struct seq_file * ...,recursed_function_seq_stop,NaN,1,Benign
4,2,13,79,recursed_function_seq_show,recursed_function_seq_show( struct seq_file * ...,158,172,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,recursed_function_seq_show( struct seq_file * ...,recursed_function_seq_show,NaN,1,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207,1,4,10,trace_init,trace_init( void),10307,10310,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,trace_init( void){\n\ttrace_event_init();\n}\n,trace_init,NaN,1,Benign
2208,2,8,25,clear_boot_tracer,clear_boot_tracer( void),10312,10327,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,clear_boot_tracer( void){\n\t/*\n\t * The defa...,clear_boot_tracer,NaN,1,Benign
2209,4,15,51,tracing_set_default_clock,tracing_set_default_clock( void),10330,10346,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,tracing_set_default_clock( void){\n\t/* sched_...,tracing_set_default_clock,NaN,1,Benign
2210,1,1,6,tracing_set_default_clock,tracing_set_default_clock( void),10348,10348,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,tracing_set_default_clock( void),tracing_set_default_clock,NaN,1,Benign


In [76]:
print(len(df_stat.columns)) 
df_stat = df_stat.dropna(axis=1, how='all')
print(len(df_stat.columns))
df_stat.head(5)

12
12


,file,line,column,category,name,msg,note,cwe,context,helpuri,severity,tool
0,data/projects/linux-rpi-6.1.y/kernel/trace/rin...,2402,NaN,NaN,NaN,NaN,NaN,Benign,/* Make sure the page didn't change since we r...,NaN,NaN,sampling
1,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,304,NaN,NaN,NaN,NaN,NaN,Benign,return val * buckets;,NaN,NaN,sampling
2,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,295,NaN,NaN,NaN,NaN,NaN,Benign,"u64 val = hist_fn_call(operand, elt, buffer, r...",NaN,NaN,sampling
3,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,283,NaN,NaN,NaN,NaN,NaN,Benign,return (u64) ilog2(roundup_pow_of_two(val));,NaN,NaN,sampling
4,data/projects/linux-rpi-6.1.y/kernel/trace/tra...,256,NaN,NaN,NaN,NaN,NaN,Benign,int str_loc = str_item & 0xffff;,NaN,NaN,sampling


In [58]:
str(df_fun[df_fun.cwe=="['CWE-120', 'CWE-120, CWE-20']"]['filename'])

'1180    data/projects/linux-rpi-6.1.y/kernel/trace/rin...\nName: filename, dtype: object'

In [82]:
import ruamel

# yaml = yaml.YAML()
# config = yaml.safe_load(open("../ext_projects.yaml"))
with open("../ext_projects.yaml") as f:
        # config = yaml.load(f, Loader=yaml.RoundTripLoader)
        doc = ruamel.yaml.load(f.read(), Loader=ruamel.yaml.RoundTripLoader)
        doc['save']['drop_dup'] = '10'
        print(ruamel.yaml.dump(doc, Dumper=ruamel.yaml.RoundTripDumper))

ruamel.yaml.preserve_quotes = True
config['save']['drop_dup'] = '10'
print(config)

# with open('file_to_edit.yaml', 'w') as f:
#         yaml.dump(config, f)

# config file for data extraction from different IoT projects:
projects: [data/projects/contiki-2.4/core, data/projects/linux-rpi-6.1.y/kernel/trace]
save:
  database: data/IoT
  statement: data/raspberryZip_statement.csv
  function: data/raspberryZip_function.csv
  override: true
  apply_guesslang: false
  threshold_lines: 5
  benign_ratio: 0.25
  seed: 0
  drop_dup: '10'

ordereddict([('projects', ['data/projects/contiki-2.4/core', 'data/projects/linux-rpi-6.1.y/kernel/trace']), ('save', ordereddict([('database', 'data/IoT'), ('statement', 'data/raspberryZip_statement.csv'), ('function', 'data/raspberryZip_function.csv'), ('override', True), ('apply_guesslang', False), ('threshold_lines', 5), ('benign_ratio', 0.25), ('seed', 0), ('drop_dup', '10')]))])


In [95]:
import ruamel
import sys

# yaml = yaml.YAML()
# config = yaml.safe_load(open("../ext_projects.yaml"))
with open("../ext_projects.yaml") as f:
    yaml = ruamel.yaml.YAML()
    yaml.indent(mapping=4, sequence=4, offset=1)
    yaml.preserve_quotes = True
    params = yaml.load(f.read())
    params['projects'].yaml_add_eol_comment('some comment', key='new_key', column=40)
    config['save']['drop_dup'] = '10'
    # params['ParentTest']['test'].yaml_add_eol_comment('some comment', key='new_key', column=40) # column is optional
    yaml.dump(params, sys.stdout)

# config file for data extraction from different IoT projects:
projects: ["data/projects/contiki-2.4/core", "data/projects/linux-rpi-6.1.y/kernel/trace"]
    # "data/projects/contiki-2.4/apps",
    # "https://sourceforge.net/projects/contiki/files/Contiki/Contiki%202.4/contiki-sky-2.4.zip/download",
    # "https://github.com/contiki-os/contiki/archive/refs/heads/master.zip",
    # "https://github.com/raspberrypi/linux/archive/refs/heads/rpi-6.1.y.zip",
    # "https://github.com/raspberrypi/firmware/archive/refs/heads/master.zip",
    # "https://github.com/SmartSecLab/ENViSEC/archive/refs/heads/main.zip",
    # "/Users/guru/research/contiki-master",
    # "/Users/guru/research/contiki-master/apps/webbrowser/",
save:
    database: "data/IoT"
    statement: "data/raspberryZip_statement.csv"
    function: "data/raspberryZip_function.csv"
    override: true # override the existing files
    apply_guesslang: false # do you want to use guess lang to classify files or use extension?
    thresh

In [126]:
import re
import os

s = """ main()
{
    int a =   10;
    int b = 20;
    int c = a + b;
}
"""

print(s.replace('/  +/g', ' '))

 main()
{
    int a =   10;
    int b = 20;
    int c = a + b;
}



In [131]:
# print([" ".join(s.split()))
# st = [re.replace() for s in s.splitlines()]
st = [x.replace('\s+', ' ') for x in s.splitlines()]
print(st)

[' main()', '{', '    int a =   10;', '    int b = 20;', '    int c = a + b;', '}']
